<a href="https://colab.research.google.com/github/nicolaspro1234/trading_auto/blob/main/trading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import random
from collections import namedtuple

In [2]:
N = 50
offset=3
random.seed(0)

def get_dummy_values(N, offset):
  res = [offset]
  value = offset
  for i in range(1, N):
    new_value = value + (offset/10) * random.random() - (offset/10) * random.random()
    res += [new_value]
    value = new_value
  return res

values = get_dummy_values(10, 3)

In [4]:
pd.DataFrame(data={'close': values})

,close
0,3.000000
1,3.025940
2,3.074437
3,3.106339
4,3.250485
5,3.218449
6,3.339477
7,3.197287
8,3.307646
9,3.285734


I want to build a pipeline that checks for me that the data follows some patter.
For example:
  - the last data point is a resistance breaker or not.
  - the last data point is the end of a head and shoulders pattern.

So I must have a pipeline object, and within it, steps that return a boolean.

# Pipeline

In [60]:
class Pipeline():
  def __init__(self, pipeline=[]):
    self.pipeline = pipeline

  def add(self, stage):
    if stage.id in self.pipeline:
      raise ValueError("A stage in pipeline already contains this stage's id")
    self.pipeline += [stage]

  def remove(self, stage_id):
    self.pipeline = [s for s in self.pipeline if s.id != stage_id]

  def run(self, data, print_metadata=False):
    for s in self.pipeline:
      b, metadata = s.function(data)
      if print_metadata:
        print(f'{s.id}: {metadata}')

      if not b:
        return False
    return True


# function returns a tuple (bool, Dict). The second element is metadata about the result.
Stage = namedtuple('Stage', ['id', 'function'])

In [61]:
s_test = Stage(id='test', function=lambda s: ( (s > 3.1).all(), (s > 3.1).sum() ))

In [62]:
p = Pipeline(pipeline=[])
p.add(s_test)
p.pipeline

[Stage(id='test', function=<function <lambda> at 0x7aa03c0293f0>)]

In [63]:
p.run(np.array(values), print_metadata=True)

test: 7


False

In [64]:
p.pipeline

[Stage(id='test', function=<function <lambda> at 0x7aa03c0293f0>)]

In [ ]:
p.remove('test')
p.pipeline

[]

# Checker fucntions

## Resistance

In [ ]:
values

[3,
 3.025940234575424,
 3.0744366837367885,
 3.1063388589122467,
 3.250484617799,
 3.218449092208197,
 3.339476901021581,
 3.1972869930943246,
 3.307645789688192,
 3.285734023667368]

In [ ]:
import numpy as np

def is_resistance(
    data,
    max_nb_historical_points,
    min_nb_historical_points,
    ratio_around_resistance,
    min_touch_occurences,
    min_points_between_touches,
    ratio_of_decrease=0.2):
    """
    Returns True if the last point is part of a resistance, else False.

    Filters the history by removing points before the latest point that exceeds
    last_point * (1 + ratio_around_resistance). Then checks for resistance with
    additional constraints on the minimum distance between touches and low-level condition.

    Arguments:
        - data:                     numpy array of points
        - max_nb_historical_points: maximum history size as number of points
        - min_nb_historical_points: minimum number of historical points in the filtered history
        - ratio_around_resistance:  ratio of tolerance around resistance level
        - min_touch_occurences:     number of times the resistance line is touched
        - min_points_between_touches: minimum points required between touches
        - ratio_of_decrease:        minimum ratio decrease from the resistance level required in history

    Returns:
        bool: True if the last point is part of a resistance, False otherwise.
    """
    # Ensure data length is sufficient
    if len(data) < max_nb_historical_points:
        raise ValueError("Insufficient data points for the specified history size.")

    # Get the last point and its tolerance threshold
    last_point = data[-1]
    tolerance_threshold = last_point * (1 + ratio_around_resistance)

    # Filter the history
    historical_data = data[-max_nb_historical_points:-1]  # Exclude the last point for resistance calculation
    crossing_indices = [i for i, point in enumerate(historical_data) if point > tolerance_threshold]

    if crossing_indices:
        # Cut history after the most recent crossing point
        historical_data = historical_data[crossing_indices[-1] + 1:]

    # Check if the filtered history meets minimum length requirement
    if len(historical_data) < min_nb_historical_points:
        return False

    # Determine the potential resistance level as the maximum value in the filtered historical data
    resistance_level = np.max(historical_data)

    # Define the tolerance range around the resistance level
    lower_bound = resistance_level * (1 - ratio_around_resistance)
    upper_bound = resistance_level * (1 + ratio_around_resistance)

    # Ensure the cut history has reached a low enough level
    required_low_level = resistance_level - resistance_level * ratio_of_decrease
    if np.min(historical_data) > required_low_level:
        return False

    # Count valid touches with min_points_between_touches enforcement
    touch_count = 0
    last_touch_index = -min_points_between_touches - 1  # Initialize as far enough back
    for i, point in enumerate(historical_data):
        if lower_bound <= point <= upper_bound:
            # Check if enough points have passed since the last touch
            if i - last_touch_index > min_points_between_touches:
                touch_count += 1
                last_touch_index = i
        elif point > upper_bound:  # Resistance is crossed
            return False

    # Check if the last point is within the tolerance range
    is_last_point_near_resistance = lower_bound <= last_point <= upper_bound

    # Return True if last point is near resistance and touch count condition is met
    return is_last_point_near_resistance and touch_count >= min_touch_occurences


## Epaule tête épaule inversée

écrire nom fonction et docstring. Puis demander à chat gpt d'écrire la fonction.

# Examples